# Aufgabe 1 & 2

In [ ]:
import numpy as np
from tqdm import tqdm
from time import sleep
import operator
import copy

In [ ]:
class NN():
    """A class to build and train a neural network used to solve the XOR problem."""
    INPUT_NEURONS = 2
    OUTPUT_NEURONS = 1

    def __init__(self, hidden_neurons=2, pop_size=10):
        """Create a 3-layer NN with two input neurons, n hidden neurons and one output neuron.
        
        hidden_neurons -- the number of neurons in the NN's hidden layer
        pop_size       -- the initial number of individuals in a population
        """
        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.h_neurons = hidden_neurons

        self.pop_size = pop_size
        
        self.w_hidden = [np.random.uniform(size=(NN.INPUT_NEURONS, self.h_neurons)) for _ in range(pop_size)]
        self.w_out = [np.random.uniform(size=(self.h_neurons, NN.OUTPUT_NEURONS)) for _ in range(pop_size)]

        self.b_hidden = [np.random.uniform(size=(1, self.h_neurons)) for _ in range(pop_size)]
        self.b_out = [np.random.uniform(size=(1, NN.OUTPUT_NEURONS)) for _ in range(pop_size)]

        self.z0s = []
        self.z1s = []

        print("Initial weights: ")
        self.print_weights()

    def ReLU(self, x):
        """ReLU activation function"""
        return x * (x > 0) 
    
    def sigmoid(self, x):
        """Sigmoid activation function"""
        return 1/(1 + np.exp(-x))
    
    def BCEL(self, y, y_hat):
        """Binary Cross-Entropy Loss
        
        y     -- the ground truth
        y_hat -- the NN's prediction
        """
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))
    
    def execute(self, x):
        """Execute the NN for a given input, return its result."""

        # For the execution, the weights of the first (index 0) individual in the population are used.
        result = self.sigmoid(self.forward_pass(0, x)[0])
        return np.round(result[0])
    
    def forward_pass(self, index, x):
        """Carry out the forward pass.
        
        index -- for using the weights of one specific individual of the population
        """
        x = np.array(x)
        z0 = np.dot(x, self.w_hidden[index]) + self.b_hidden[index]
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden, self.w_out[index]) + self.b_out[index]
        return z1, z0


    def neuro_train(self, print_details = True, crossover = False):
        """Train the NN's weights using evolutionary algorithms."""
        epoch = 1
        # The weights of the first (index 0) individual in the population are used to check the NN's accuracy.
        while self.accuracy(0)[0] != 1:
            if print_details: print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))

            w_out = self.copy(self.w_out)
            b_out = self.copy(self.b_out)
            w_hidden = self.copy(self.w_hidden)
            b_hidden = self.copy(self.b_hidden)
            
            if crossover:
                w_out = self.crossover(w_out)
                b_out = self.crossover(b_out)
                w_hidden = self.crossover(w_hidden)
                b_hidden = self.crossover(b_hidden)

            self.mutation(w_out)
            self.mutation(b_out)
            self.mutation(w_hidden)
            self.mutation(b_hidden)
            
            self.w_out += w_out
            self.w_hidden += w_hidden
            self.b_out += b_out
            self.b_hidden += b_hidden

            f_list = self.fitness()
            self.selection(f_list)

            epoch += 1

        print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))
        self.print_weights()
    
    def copy(self, weight):
        """Create a copy of a given weight (list)."""
        new_weight = []
        for w in weight:
            new_weight.append(w.copy())
        return new_weight
    
    def fitness(self):
        """Calculate the fitness of each individual in the population, using the respective loss."""
        f_list = []
        for i in range(len(self.w_hidden)):
            acc, loss = self.accuracy(i)
            f_list += [1-acc+loss]
        return f_list

    def selection(self, f_list, use_prop_slct = False):
        """Select a subset of the population to be kept.
        
        f_list        -- list of fitness values for every individual
        use_prop_slct -- whether to use fitness proportional selection (default False)
        """
        dic = dict()
        for i in range(len(f_list)):
            dic.update([(i,f_list[i])])
        sortedPop = sorted(dic.items(), key=operator.itemgetter(1), reverse=False)
        last = sortedPop[self.pop_size:]
        last.sort()
        c = 0
        for index in last:
            del self.w_out[index[0]-c]
            del self.w_hidden[index[0]-c]
            del self.b_out[index[0]-c]
            del self.b_hidden[index[0]-c]
            c += 1
        

    def crossover(self, weight):
        """Perform a crossover on a given weight (list)."""
        w = np.zeros(shape=(len(weight), weight[0].shape[0], weight[0].shape[1]))
        lis = [i for i in range(len(weight))]
        np.random.shuffle(lis)
        for k in range(0, len(weight), 2):
            # prevent overflow in case of odd number of individuals
            if len(weight) % 2 != 0 and k == len(weight)-1: break
            for i in range(0, weight[0].shape[0]):
                for j in range(0, weight[0].shape[1]):
                    if np.random.sample() > 0.5:
                        w[lis[k]][i][j] = weight[lis[k+1]][i][j]
                        w[lis[k+1]][i][j] = weight[lis[k]][i][j]
                    else:
                        w[lis[k]][i][j] = weight[lis[k]][i][j]
                        w[lis[k+1]][i][j] = weight[lis[k+1]][i][j]
        return list(w)

    def mutation(self, weight):
        """Mutate a given weight (list) by adding values sampled from a normal destribution onto each entry."""
        for k in range(len(weight)):
            for i in range(len(weight[k])):
                for j in range(len(weight[k][i])):
                    weight[k][i][j] += np.random.normal(0,0.1)


    def print_weights(self):
        """Print the weights of the NN."""

        print("W_hidden: " + str(self.w_hidden[0])) 
        print("B_hidden: " + str(self.b_hidden[0])) 
        print("W_out: " + str(self.w_out[0])) 
        print("B_out: " + str(self.b_out[0]), end="\n\n")
        pass

    def accuracy(self, index):
        """Calculate the accuracy and loss of the NN for a specific individual.
        
        index -- for using the weights of one specific individual of the population
        """
        counter = 0
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        
        loss = 0
        for i in range(len(self.data)):
            loss += self.BCEL(self.target[i], self.sigmoid(self.forward_pass(index, self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
    

In [ ]:
nn = NN(4, pop_size=10)

In [ ]:
nn.neuro_train(print_details=True, crossover=True)

In [ ]:
for i in nn.data:
    print(nn.execute(i))

Bei "Fitness Proportionate Selection" ist die Wahrscheinlichkeit eines jeden Individuums, selektiert zu werden, mit unserer Fitness-Funktion ziemlich klein. "Elitist Selection" bringt wiederum Konstanz in die Populationsgröße.

# Aufgabe 3


In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2, 1)),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

model.compile(loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
data = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

target = np.array([[0],[1],[1],[0]])

In [ ]:
model.fit(data, target, epochs=300)

In [ ]:
model.evaluate(data,  target, verbose=2)

Von den drei Verfahren wird das Training bis hin zu einer Accuracy von 1 im Allgemeinen am schnellsten von dem Modell, das evolutionäre Algorithmen nutzt, durchgeführt. Am langsamsten ist die manuelle Variante. 